<a href="https://colab.research.google.com/github/uteyechea/crime-prediction-using-artificial-intelligence/blob/master/validate_rnn_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import gc
import sys

import pandas as pd
from scipy import stats

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path='/content/drive/My Drive/Colab Notebooks/crime_prediction'

sys.path.append(path+'/libs')
import temporal_correlation as tc


Mounted at /content/drive


In [9]:
prediction_file_path=os.path.join(path,'data','prediction','series_sample.txt')
prediction_file=pd.read_csv(prediction_file_path,sep=',')

In [11]:
prediction_file.isnull().values.any()

False

In [14]:
training_data_path=os.path.join(path,'data','training','theft2.csv')
training_file=pd.read_csv(training_data_path,sep=',')

In [17]:
training_file

,-0.28868455330176296
0,-1.043577
1,-1.043577
2,-0.288685
3,-1.295207
4,-1.295207
...,...
887,0.214577
888,0.969469
889,3.234145
890,0.969469


In [18]:
type(prediction_file)

pandas.core.frame.DataFrame

In [19]:
type(training_file)

pandas.core.frame.DataFrame

In [21]:
prediction_file.shape

(2555, 1)

In [23]:
training_file.shape

(892, 1)

In [28]:
def series_window(dataframe,end,periods,column):
  if type(column)==str:
    column=dataframe.columns.get_loc(column)
  return dataframe.iloc[end-periods:end,column]

def series_correlation(series1,series2):
  #assert something?
  series1=series1.reset_index(drop=True) #Better find a way to simplify this procedure
  series2=series2.reset_index(drop=True) #Better find a way to simplify this procedure
  ro=series1.corr(series2) 
  return ro 

def correlated_series_timestamp(dataframe,end,periods,column,min_correlation):
  timestamps=[]
  series2=series_window(dataframe,end,periods,column)
  for epoch in reversed(range(2*periods,end)):
    series1=series_window(dataframe,epoch-periods,periods,column)
    correlation=series_correlation(series1,series2)
    if correlation >= min_correlation:
      #record datetime value. We will use this datetime value to generate all sequences that will go as input to the RNN
      #print(dataframe.index[epoch]) 
      timestamps.append(dataframe.index[epoch])
  return timestamps


In [29]:
get_correlated_endof_sequence_timestamp(prediction_file,training_file,len(training_file),n=10)

[]